In [1]:
# Try the PyPI version instead of GitHub
%pip install -q --no-cache-dir pyterrier-t5

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

/home/killdollar/btp/Information-Retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
indexer = pt.IterDictIndexer('./indices/msmarco-passage')
index = indexer.path

Java started (triggered by TerrierIndexer.__init__) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
bm25 = pt.terrier.Retriever(index, wmodel="BM25", num_results=100)

In [4]:
# Use judged dataset only for topics/qrels
ds_topics = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')

In [5]:
# Use full corpus for text lookup (matches the ids in your index)
ds_text = pt.get_dataset('irds:msmarco-passage')

In [9]:
# Get topics from judged set
topics = ds_topics.get_topics()

In [ ]:

# Create a custom text fetcher that only gets text for retrieved documents
def get_text_for_docnos(docnos, dataset):
    """Efficiently get text for specific document IDs"""
    doc_texts = {}
    
    # Convert all docnos to the format we'll try
    docno_variants = {}
    for docno in docnos:
        str_docno = str(docno)
        docno_variants[str_docno] = docno
        # Also try integer version if it's numeric
        if str_docno.isdigit():
            docno_variants[int(str_docno)] = docno
    
    # Only iterate through corpus until we find all needed documents
    needed_ids = set(docno_variants.keys())
    found_count = 0
    
    corpus_iter = dataset.get_corpus_iter()
    for doc in corpus_iter:
        doc_id = doc['docno']
        
        # Check if this document is one we need
        if doc_id in needed_ids:
            original_docno = docno_variants[doc_id]
            doc_texts[original_docno] = doc['text']
            found_count += 1
            
            # Stop early if we found all documents we need
            if found_count >= len(docnos):
                break
    
    # Fill in missing documents
    for docno in docnos:
        if docno not in doc_texts:
            doc_texts[docno] = "Document text not available"
    
    return doc_texts

# Create a transformer that adds text efficiently
def add_text_efficiently(df):
    """Add text to dataframe for only the documents in this batch"""
    docnos = df['docno'].tolist()
    doc_texts = get_text_for_docnos(docnos, ds_text)
    
    df = df.copy()
    df['text'] = df['docno'].map(doc_texts)
    return df

# Create efficient text retriever
text_retriever = bm25 >> pt.apply.generic(add_text_efficiently)

# Test it with a single query
test_results = text_retriever.search(topics.iloc[0]['query'])
print(f"Retrieved {len(test_results)} documents")
print(f"Text available for {sum(1 for t in test_results['text'] if t != 'Document text not available')} documents")
print("\nSample results:")
print(test_results[['docno', 'score', 'text']].head(3))


















































































































































































































































































msmarco-passage documents: 100%|█████████▉| 8803136/8841823 [00:28<00:00, 309510.82it/s]

Retrieved 100 documents
Text available for 100 documents

Sample results:
     docno      score                                               text
0  8182161  31.156444  A: While goldfish can grow up to 18 inches in ...
1  6139386  31.032333  A: The conditions goldfish are kept in plus th...
2  3288600  30.953843  A goldfish will grow to the depth of the water...


In [30]:
# Process each query with your working text retrieval method
import pandas as pd

# Fix: Use ds_topics instead of dataset
topics = ds_topics.get_topics()
all_results = []

for idx, row in topics.iterrows():
    query_id = row['qid']
    query_text = row['query']
    
    print(f"Processing query {idx+1}/{len(topics)}: {query_text}")
    
    # Step 1: Get initial results from BM25
    initial_results = bm25.search(query_text)
    
    # Step 2: Fetch document text using your working method
    docnos = initial_results['docno'].tolist()
    doc_texts = get_text_for_docnos(docnos, ds_text)
    
    # Add text to results - CRITICAL MISSING LINE
    initial_results['text'] = initial_results['docno'].map(doc_texts)
    
    # Step 3: Apply MonoT5 reranking (now with text available)
    try:
        reranked_results = monoT5.transform(initial_results)
        # Sort by MonoT5 score and take top 10
        top_results = reranked_results.sort_values('score', ascending=False).head(10)
        print(f"MonoT5 reranking successful")
    except Exception as e:
        print(f"MonoT5 reranking error: {e}")
        # Fallback to BM25 ranking
        top_results = initial_results.head(10)
    
    # Add query information to results
    top_results['qid'] = query_id
    top_results['query'] = query_text
    
    all_results.append(top_results)
    
    # Show progress
    if idx == 0:
        print("\nSample results for first query:")
        print(top_results[['docno', 'score']].head(3))

# Combine all results
combined_results = pd.concat(all_results) if all_results else pd.DataFrame()

# Save results to CSV
combined_results.to_csv('monot5_results.csv', index=False)

print(f"\nRetrieved {len(combined_results)} documents across {len(topics)} queries")
print(f"Results saved to monot5_results.csv")

# Show summary of results
print("\nTop documents for first few queries:")
print(combined_results.groupby('qid').head(1)[['qid', 'docno', 'score']].head(5))

Processing query 1/43: do goldfish grow

















































































































































































































































































msmarco-passage documents: 100%|█████████▉| 8803136/8841823 [00:28<00:00, 311530.92it/s]


























monoT5: 100%|██████████| 25/25 [00:18<00:00,  1.32batches/s]


MonoT5 reranking successful

Sample results for first query:
      docno     score
26  8182159 -0.003995
17  1960255 -0.007046
2   3288600 -0.013163
Processing query 2/43: what is wifi vs bluetooth










































































































































































































































































msmarco-passage documents:  94%|█████████▍| 8301973/8841823 [00:26<00:01, 309999.64it/s]


























monoT5: 100%|██████████| 25/25 [00:19<00:00,  1.28batches/s]


MonoT5 reranking successful
Processing query 3/43: why did the us volunterilay enter ww1


...

MonoT5 reranking successful
Processing query 42/43: hydrogen is a liquid below what temperature











































































































































































































































































msmarco-passage documents:  99%|█████████▊| 8712732/8841823 [00:26<00:00, 325690.31it/s]


























monoT5: 100%|██████████| 25/25 [00:19<00:00,  1.25batches/s]


MonoT5 reranking successful
Processing query 43/43: difference between a mcdouble and a double cheeseburger








































































































































































































































































msmarco-passage documents:  99%|█████████▉| 8795495/8841823 [00:26<00:00, 332601.47it/s]


























monoT5: 100%|██████████| 25/25 [00:17<00:00,  1.45batches/s]

MonoT5 reranking successful

Retrieved 425 documents across 43 queries
Results saved to monot5_results.csv

Top documents for first few queries:
        qid    docno     score
26   156493  8182159 -0.003995
10  1110199  8160520 -0.030058
47  1063750  7952971 -0.004671
13   130510  8612903 -0.003900
13   489204  8737051 -0.057302
